In [10]:
import sys
from timeit import default_timer as timer

sys.path.append('../pmm/')

import inputs
from pmm.pmm import convert2Universe, split_qc_solv, calc_el_field_pot, calc_pmm_matrix, rotate_dip_matrix
import conversions
import MDAnalysis as mda
import numpy as np
#import numpy.linalg 
import nglview as nv

In [11]:
# gather the electronic properties
start = timer()
gauss_file = 'thioindigo/thio_pbe0_esp{}_x180.log'
n_el_states = 16
pmm_inputs = inputs.get_tot_input_gauss(gauss_file, n_el_states)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(pmm_inputs)

qc_geom = convert2Universe(pmm_inputs['geometry'])
# shift origin to center of mass
cdm_ref = qc_geom.atoms.center_of_mass()
qc_geom.atoms.positions = qc_geom.atoms.positions - cdm_ref

print(qc_geom)


inputs da Gaussian ci mette:  0.03302109999640379
<Universe with 28 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [6]:
# Read the MD trajectory
start = timer()
traj = mda.Universe('thioindigo/thioindigo_trans_etoh_md.tpr', 'thioindigo/thio_etoh_center_10ps.xtc')
end = timer()
print('leggere xtc ci mette: ', end - start)
w = nv.show_mdanalysis(traj)
w

#qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
#print(qc_traj)
#w = nv.show_mdanalysis(solv_traj)
#w

leggere xtc ci mette:  0.10023150000051828


NGLWidget(max_frame=10)

In [7]:
# MD-PMM calculation

start = timer()
el_field_traj = []
potential_traj = []
rot_dip_matrix_traj = []
pmm_traj = []
for ts in traj.trajectory:
    # Divide the trajectory in QC and solvent.
    # NOTE: the indexes for the QC are inclusive of the extremes.
    qc_traj, solv_traj = split_qc_solv(traj, '1:28')
    #print(pmm.rotate_dip_matrix(pmm_inputs['dip_matrix'], qc_traj, qc_geom))

    # Calculate the center of mass of QC
    cdm = qc_traj.center_of_mass()
    print('cdm: ', cdm)
    #print(solv_traj.atoms.positions)

    # Calculate the el_field
    el_field, potential = calc_el_field_pot(solv_traj.atoms.positions, solv_traj.atoms.charges, cdm)
    el_field_traj.append(el_field)
    potential_traj.append(potential)
    #print(el_field, potential)

    # Rotate the dip_matrix using mwsfit
    rot_dip_matrix = rotate_dip_matrix(pmm_inputs['dip_matrix'], qc_traj, qc_geom)
    rot_dip_matrix_traj.append(rot_dip_matrix)

    pmm_matrix = calc_pmm_matrix(pmm_inputs['energies'], rot_dip_matrix, el_field, potential)
    #print(pmm_matrix)
    pmm_traj.append(np.linalg.eig(pmm_matrix))
end = timer()
print('PMM ci mette: ', end - start)
print(len(pmm_traj), pmm_traj[0][0].shape)


AttributeError: module 'pmm' has no attribute 'split_qc_solv'

In [ ]:
# Check the electric field:
for frame in el_field_traj:
    print(*frame)

In [ ]:
# Check potential
for frame in potential_traj:
    print(frame)

In [ ]:
# Check the rotated dip matrix
for frame in rot_dip_matrix_traj:
    print(frame)